In [7]:
# Importando as bibliotecas necessárias
import os
import cv2
import numpy as np
from deepface import DeepFace
from tqdm import tqdm
import pickle

In [8]:
# --- DEFINIÇÃO DOS CAMINHOS ---
# Como nosso notebook está na pasta 'notebook/', usamos '../' para "subir um nível"
# e depois entrar na pasta 'data'.
caminho_dados_originais = '../data/'
caminho_faces_recortadas = '../data/faces_recortadas/'

# --- CONFIGURAÇÃO DO DETECTOR FACIAL (OpenCV DNN) ---
# O OpenCV possui um detector de faces pré-treinado muito eficiente.
# Precisamos de dois arquivos para carregá-lo:
# 1. O arquivo .prototxt, que define a arquitetura da rede.
# 2. O arquivo .caffemodel, que contém os pesos pré-treinados.
#
# Você precisará baixar esses dois arquivos e colocá-los na sua pasta
# principal 'Reconhecimento_Facial/'.
#
# Você pode encontrá-los facilmente no GitHub oficial do OpenCV:
# Arquivo 1: deploy.prototxt.txt
# Arquivo 2: res10_300x300_ssd_iter_140000.caffemodel
#
# Link para a pasta com os arquivos:
# https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector

# Carregando o modelo do disco
print("Carregando o modelo de detecção facial...")

caminho_prototxt = '../deploy.prototxt'
caminho_modelo = '../res10_300x300_ssd_iter_140000.caffemodel'

detector = cv2.dnn.readNetFromCaffe(caminho_prototxt, caminho_modelo)
print("Modelo carregado com sucesso!")


# Criar a pasta de saída se ela não existir
if not os.path.exists(caminho_faces_recortadas):
    os.makedirs(caminho_faces_recortadas)
    print(f"Diretório criado em: {caminho_faces_recortadas}")

Carregando o modelo de detecção facial...
Modelo carregado com sucesso!


In [4]:
# Iterar sobre cada pessoa (cada subpasta em 'data/')
for nome_pessoa in os.listdir(caminho_dados_originais):
    caminho_pessoa = os.path.join(caminho_dados_originais, nome_pessoa)

    # Ignorar arquivos que não são diretórios (e a nossa pasta de recortes)
    if not os.path.isdir(caminho_pessoa) or nome_pessoa == 'faces_recortadas':
        continue

    print(f"Processando imagens de: {nome_pessoa}")

    # Iterar sobre cada imagem da pessoa
    for nome_arquivo in os.listdir(caminho_pessoa):
        caminho_imagem = os.path.join(caminho_pessoa, nome_arquivo)
        
        # Carregar a imagem
        imagem = cv2.imread(caminho_imagem)
        if imagem is None:
            print(f"  - Erro ao ler a imagem: {nome_arquivo}")
            continue
        
        (h, w) = imagem.shape[:2]

        # Pré-processamento: criar um 'blob' da imagem para a rede neural
        blob = cv2.dnn.blobFromImage(cv2.resize(imagem, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

        # Passar o blob pela rede para obter as detecções
        detector.setInput(blob)
        deteccoes = detector.forward()

        # Encontrar a detecção com a maior confiança (o rosto mais provável)
        best_detection_index = np.argmax(deteccoes[0, 0, :, 2])
        confianca = deteccoes[0, 0, best_detection_index, 2]
        
        # Filtrar detecções fracas
        if confianca > 0.8: # Usamos um limiar de 80% de confiança
            # Calcular as coordenadas (x, y) da caixa delimitadora do rosto
            box = deteccoes[0, 0, best_detection_index, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Garantir que a caixa não saia dos limites da imagem
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)

            # Recortar o rosto
            rosto = imagem[startY:endY, startX:endX]
            
            if rosto.size != 0:
                # Salvar o rosto recortado
                nome_saida = f"{nome_pessoa}_{nome_arquivo}"
                caminho_saida = os.path.join(caminho_faces_recortadas, nome_saida)
                cv2.imwrite(caminho_saida, rosto)
                print(f"  - Rosto salvo em: {nome_saida}")
            else:
                print(f"  - Rosto detectado mas recorte resultou em imagem vazia: {nome_arquivo}")
        else:
            print(f"  - Nenhum rosto encontrado com confiança suficiente em: {nome_arquivo}")


print("\nProcessamento concluído!")

Processando imagens de: Kaik
  - Rosto salvo em: Kaik_1.jpg
  - Rosto salvo em: Kaik_2.jpg
  - Rosto salvo em: Kaik_3.jpg
  - Rosto salvo em: Kaik_4.jpg
  - Rosto salvo em: Kaik_5.jpg
Processando imagens de: Rick
  - Rosto salvo em: Rick_1.jpg
  - Rosto salvo em: Rick_2.jpg
  - Rosto salvo em: Rick_3.jpg
  - Rosto salvo em: Rick_4.jpg
  - Rosto salvo em: Rick_5.jpg

Processamento concluído!


In [5]:
# --- CARREGANDO O MODELO FACENET USANDO DEEPFACE ---
print("Construindo o modelo FaceNet via DeepFace...")
# Como o modelo já foi baixado, esta chamada será quase instantânea.
model = DeepFace.build_model('Facenet') 
print("Modelo FaceNet construído com sucesso!")

# --- PROCESSANDO OS ROSTOS E GERANDO EMBEDDINGS ---
known_embeddings = []
known_names = []
caminho_faces_recortadas = '../data/faces_recortadas/'

print("\nIniciando a geração de embeddings...")
for nome_arquivo in tqdm(os.listdir(caminho_faces_recortadas), desc="Processando rostos"):
    nome_pessoa = nome_arquivo.split('_')[0]
    caminho_rosto = os.path.join(caminho_faces_recortadas, nome_arquivo)
    
    try:
        # --- CÓDIGO CORRIGIDO AQUI ---
        # Removido o argumento 'model=model'
        embedding_obj = DeepFace.represent(
            img_path = caminho_rosto, 
            model_name = 'Facenet',
            enforce_detection = False
        )
        embedding = embedding_obj[0]['embedding']
        
        known_embeddings.append(embedding)
        known_names.append(nome_pessoa)
        
    except Exception as e:
        print(f"\nErro ao processar {nome_arquivo}: {e}")

print(f"\n{len(known_embeddings)} embeddings gerados com sucesso.")

# --- SALVANDO O BANCO DE DADOS DE EMBEDDINGS ---
dados_embeddings = {"embeddings": np.array(known_embeddings), "names": known_names}
caminho_saida_pickle = '../data/embeddings.pickle'
with open(caminho_saida_pickle, 'wb') as f:
    pickle.dump(dados_embeddings, f)

print(f"\nBanco de dados de embeddings salvo com sucesso em: {caminho_saida_pickle}")

Directory  C:\Users\pcwin /.deepface created
Directory  C:\Users\pcwin /.deepface/weights created
Construindo o modelo FaceNet via DeepFace...
facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: C:\Users\pcwin\.deepface\weights\facenet_weights.h5
100%|██████████████████████████████████████████████████████████████████████████████| 92.2M/92.2M [02:46<00:00, 554kB/s]


Modelo FaceNet construído com sucesso!

Iniciando a geração de embeddings...


Processando rostos: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.58it/s]


10 embeddings gerados com sucesso.

Banco de dados de embeddings salvo com sucesso em: ../data/embeddings.pickle
